In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import emoji

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier


import re
import emoji
import pandas as pd
from nltk.corpus import stopwords
from wordsegment import load, segment


import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/vishalthehuman/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [35]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import StackingClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np

In [36]:
# ---------------------------
# 1. Load and Combine Datasets
# ---------------------------
train_df = pd.read_csv("original/train.csv")
dev_df = pd.read_csv("original/dev.csv")
df = pd.concat([train_df, dev_df], ignore_index=True)
df

,id,text,label
0,3268,Indha ariya kandupidippin moolam neenga solla ...,0
1,6239,@vijayakumarp7959 unmai therincha nee pesu,0
2,5859,Inga erukka yella dev... boys vadakkan vadakk...,1
3,3519,பீகாரி பிரசாந்த் கிஷோரிடம் கொடுத்த 350 கோடியை ...,1
4,5136,Mumbai Bangalore la 80% percentage outsiders,1
...,...,...,...
6294,5860,This is done by those who want chaos in Tamil ...,0
6295,1441,"வடமாநிலத்தவருக்கு வேலை வாய்ப்பு கொடுப்பதை ,பணி...",0
6296,3253,நம்ம மேல எந்த தப்பும் இல்லை.... ஏன் என்றால் நா...,0
6297,4295,தம்பி உத்திரபிரதேசம் டெல்லி எல்லையில் உள்ள 90%...,1


In [37]:
df["label"].value_counts()

label
0    3900
1    2399
Name: count, dtype: int64

In [38]:
# -------------------------------
# Text Preprocessing
# -------------------------------
# Function to clean each text
# Load word segmentation model
load()

# Load stopwords
tamil_stopwords = set(stopwords.words('tamil'))
english_stopwords = set(stopwords.words('english'))

# -------------------------------
# Text Preprocessing
# -------------------------------

def preprocess_text(text):
    # Extract hashtags
    hashtags = re.findall(r"#\w+", text)

    # Process hashtags (convert to words)
    processed_hashtags = []
    for tag in hashtags:
        tag = tag.lstrip("#")  # Remove #
        words = segment(tag)   # Split compound words
        processed_hashtags.append(" ".join(words))  # Convert to sentence-like format

    # Remove hashtags from main text
    text = re.sub(r"#\w+", "", text)

    # Replace mentions with a placeholder
    text = re.sub(r'@\w+', '<USER>', text)
    
    # Convert emojis to text
    text = emoji.demojize(text, delimiters=(" ", " "))  # Adds spaces around emoji descriptions
    
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()

    # Merge cleaned text with processed hashtags
    final_text = text + " " + " ".join(processed_hashtags)
    final_text = final_text.replace("-", " ")  # Remove double spaces
    return final_text

# Function to remove stopwords
def remove_stopwords(text):
    words = text.split()
    filtered = [word for word in words if word.lower() not in english_stopwords and word.lower() not in tamil_stopwords]
    return ' '.join(filtered)

# Apply preprocessing
df['clean_text'] = df['text'].apply(preprocess_text).apply(remove_stopwords)

# Show sample processed data
df.sample(50)


,id,text,label,clean_text
5931,2721,டே நீங்க வீடியோ எடுத்து அவனை பெரிய ஆளாக ஆகிறா...,1,டே நீங்க வீடியோ எடுத்து அவனை பெரிய ஆளாக ஆகிறான...
4096,936,Capitalism. Less labor and more profit for own...,0,Capitalism. Less labor profit owners. Yet peop...
2661,6639,@user-pm3ol6oc9z perukku pinnadi perumaiya po...,1,<USER> pm3ol6oc9z perukku pinnadi perumaiya po...
4758,3282,North Indian viratta vendum,1,North Indian viratta vendum
1816,1671,Maarvaad le irndhu vandhu ingu vandhu koluppu ...,1,Maarvaad le irndhu vandhu ingu vandhu koluppu ...
5993,6073,பள்ளி பசங்க.. MBC கோட்டா 32% போலீஸ்காரன் எல்லா...,1,பள்ளி பசங்க.. MBC கோட்டா 32% போலீஸ்காரன் இவனுங...
421,3459,Madurai muthu and seman is response for this p...,1,Madurai muthu seman response put person
5983,2569,Thavidyaaluku porntha passga vadakanss 😂,1,Thavidyaaluku porntha passga vadakanss face_wi...
3481,5471,முன்பு ஆங்கிலேயர்கள் எப்படி வந்து ஆக்கிரமித்து...,1,முன்பு ஆங்கிலேயர்கள் எப்படி ஆக்கிரமித்து அதிகா...
4396,4239,சிவசேனா மாதிரி இருந்தா இவனுங்க இந்தமாதிரி பேசம...,1,சிவசேனா மாதிரி இருந்தா இவனுங்க இந்தமாதிரி பேசம...


In [39]:
df.shape

(6299, 4)

In [40]:
X = df['clean_text']

vectorizer = TfidfVectorizer(
    max_features=7000,
    ngram_range=(1,2),
    analyzer='word',
    token_pattern=r"(?u)\b\w+\b"
)

X_vectorized = vectorizer.fit_transform(X)
X_vectorized = X_vectorized.toarray()
X_vectorized.shape

(6299, 7000)

In [41]:
X_vectorized.shape

(6299, 7000)

In [42]:
import numpy as np

data = np.load("muril_large_embeddings_before_aug.npy")  # Replace "file.npy" with your actual file name
type(data)

numpy.ndarray

In [43]:
X_vectorized.shape

(6299, 7000)

In [44]:
X_final = np.zeros((X_vectorized.shape[0], X_vectorized.shape[1] + data.shape[1]))
X_final[:, :X_vectorized.shape[1]] = X_vectorized
X_final[:, X_vectorized.shape[1]:] = data
X_final.shape

(6299, 8024)

In [45]:
df.shape

(6299, 4)

In [48]:
y = df['label']
y

0       0
1       0
2       1
3       1
4       1
       ..
6294    0
6295    0
6296    0
6297    1
6298    0
Name: label, Length: 6299, dtype: int64

In [49]:
model = VotingClassifier(estimators=[
    ('rf', RandomForestClassifier(class_weight='balanced')),
    ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='logloss', scale_pos_weight=2.0)),
    ('lgb', LGBMClassifier(class_weight='balanced'))
], voting='soft')

model.fit(X_final, y)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [20:13:32] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[LightGBM] [Info] Number of positive: 2399, number of negative: 3900
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.195864 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 309642
[LightGBM] [Info] Number of data points in the train set: 6299, number of used features: 2504
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000


VotingClassifier(estimators=[('rf',
                              RandomForestClassifier(class_weight='balanced')),
                             ('xgb',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric='logloss',
                                            feature_types=None, gamma=None,
                                            grow_policy=None,
                                            i...
                                            interaction_constraints=None,
                                            learning_rate=None, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=None,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=None, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=None, ...)),
                             ('lgb', LGBMClassifier(class_weight='balanced'))],
                 voting='soft')

In [50]:
import pandas as pd

test_df = pd.read_csv("original/test.csv")
test_df

,id,text
0,4594,Humanity paatha intha slag countries prblm var...
1,3593,Avanga Holi kondada oruku poranga da
2,2533,Avangala avanga ooruke anupura vazhiya parunga.
3,6975,பைத்தியமாடா நீ.. நான் சோழிய வேளாளர். நான் என் ...
4,6011,Caste is there parayan think to marry higheru...
...,...,...
1571,6360,Ivlo varalaru pesarelada kena...aprm ethuku da...
1572,6208,@dineshkumar6695 தம்பி. யார் பேச்சையும் கேட...
1573,6871,பார்ப்பானும் பறையனும் ஒன்னு பழமொழி உள்ளது 😁😂
1574,442,நம்ம இடத்தை காப்பாத்திக்கிரது பேரு ரவுடிசம்னா ...


In [51]:
test_df['clean_text'] = test_df['text'].apply(preprocess_text).apply(remove_stopwords)
test_df.sample(50)

,id,text,clean_text
52,7577,யாரு ஜெயித்தது கடைசியில் சொல்லவே இல்லையே ஓய்😂😂,யாரு ஜெயித்தது கடைசியில் சொல்லவே இல்லையே ஓய் f...
836,5999,Tamillarkalai adithal yarum keppathillai,Tamillarkalai adithal yarum keppathillai
983,1473,Vaay udharuthaan naam thamizhargal.appa appa i...,Vaay udharuthaan naam thamizhargal.appa appa i...
88,5989,ஒரே ஒரு நியூஸ் சேனல் இருக்கும் போது கரெக்டாக ந...,நியூஸ் சேனல் கரெக்டாக நியூஸ் உண்மையான நியூஸ் ச...
559,533,@keralanaturelover196 Hello excuse me. Pleas...,<USER> Hello excuse me. Please read fellow wri...
1011,2671,Dai India glitz...Thiruppur event paatha illa ...,Dai India glitz...Thiruppur event paatha illa ...
794,2118,More Tamil population must start stores and bu...,Tamil population must start stores businesses ...
1185,6737,They have their own identity..let them have..a...,identity..let have..at end Tamil PPL . folded_...
503,2034,"நீங்கள் தும்பிகள்,,,,அவன் பேசுர தமிழ் உனக்கு த...","நீங்கள் தும்பிகள்,,,,அவன் பேசுர தமிழ் உனக்கு த..."
1243,5374,ellaaraiyum manisana paarunka vadamanilanu p...,ellaaraiyum manisana paarunka vadamanilanu pes...


In [52]:
X = test_df['clean_text']

vectorizer = TfidfVectorizer(
    max_features=7000,
    ngram_range=(1,2),
    analyzer='word',
    token_pattern=r"(?u)\b\w+\b"
)

X_vectorized = vectorizer.fit_transform(X)
X_vectorized = X_vectorized.toarray()
X_vectorized.shape

(1576, 7000)

In [53]:
import numpy as np

data = np.load("muril_large_embeddings_test.npy")  # Replace "file.npy" with your actual file name

In [54]:
data.shape

(1576, 1024)

In [55]:
X_final = np.zeros((X_vectorized.shape[0], X_vectorized.shape[1] + data.shape[1]))
X_final[:, :X_vectorized.shape[1]] = X_vectorized
X_final[:, X_vectorized.shape[1]:] = data
X_final.shape

(1576, 8024)

In [ ]:
predictions = model.predict(X_final)
predictions

array([0, 0, 0, ..., 0, 0, 0])

In [57]:
test_df['predictions'] = predictions

In [58]:
test_df.drop(columns=['clean_text'], inplace=True)
test_df.drop(columns=['text'], inplace=True)

In [60]:
test_df["predictions"].value_counts()  

predictions
0    1058
1     518
Name: count, dtype: int64

In [61]:
test_df.to_csv("muril_tfidf_7000.csv", index=False, header=False)